In [ ]:
!pip install sagemaker --upgrade

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
import uuid

bucket = sagemaker_session.default_bucket()
checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoint-{}'.format(bucket, checkpoint_suffix)

print('Checkpointing Path: {}'.format(checkpoint_s3_path))

In [ ]:
!pygmentize train.py

In [ ]:
use_spot_instances = True
max_run=600
max_wait = 1200 if use_spot_instances else None

In [ ]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(base_job_name='pytorch-smdataparallel-histopathology-mil',
                        entry_point='train.py',
                        role=role,
                        framework_version='1.8.0',
                        py_version='py36',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=2,
                        # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                                 }
                                          }
                                      },
                        checkpoint_s3_uri=checkpoint_s3_path,
                        debugger_hook_config=False,
                        use_spot_instances=use_spot_instances,
                        max_run=max_run,
                        max_wait=max_wait)

In [ ]:
estimator.fit()